## Predicting loan e-signing– Binomial Classification

In [ ]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
import pandas_profiling as pp
import warnings
warnings.filterwarnings('ignore')


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving P39-Financial-Data.csv to P39-Financial-Data (1).csv


In [ ]:
# import data

data = pd.read_csv('P39-Financial-Data.csv')

####  Data Review

In [ ]:
data.head()

,entry_id,age,pay_schedule,home_owner,income,months_employed,years_employed,current_address_year,personal_account_m,personal_account_y,has_debt,amount_requested,risk_score,risk_score_2,risk_score_3,risk_score_4,risk_score_5,ext_quality_score,ext_quality_score_2,inquiries_last_month,e_signed
0,7629673,40,bi-weekly,1,3135,0,3,3,6,2,1,550,36200,0.737398,0.903517,0.487712,0.515977,0.580918,0.380918,10,1
1,3560428,61,weekly,0,3180,0,6,3,2,7,1,600,30150,0.738510,0.881027,0.713423,0.826402,0.730720,0.630720,9,0
2,6934997,23,weekly,0,1540,6,0,0,7,1,1,450,34550,0.642993,0.766554,0.595018,0.762284,0.531712,0.531712,7,0
3,5682812,40,bi-weekly,0,5230,0,6,1,2,7,1,700,42150,0.665224,0.960832,0.767828,0.778831,0.792552,0.592552,8,1
4,5335819,33,semi-monthly,0,3590,0,5,2,2,8,1,1100,53850,0.617361,0.857560,0.613487,0.665523,0.744634,0.744634,12,0


In [ ]:
def data_view(df):
    print(df.head())
    print('*'*25)
    print(df.info())
    print('*'*25)
    print(df.describe())

In [ ]:
data_view(data)

   entry_id  age  ... inquiries_last_month  e_signed
0   7629673   40  ...                   10         1
1   3560428   61  ...                    9         0
2   6934997   23  ...                    7         0
3   5682812   40  ...                    8         1
4   5335819   33  ...                   12         0

[5 rows x 21 columns]
*************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17908 entries, 0 to 17907
Data columns (total 21 columns):
entry_id                17908 non-null int64
age                     17908 non-null int64
pay_schedule            17908 non-null object
home_owner              17908 non-null int64
income                  17908 non-null int64
months_employed         17908 non-null int64
years_employed          17908 non-null int64
current_address_year    17908 non-null int64
personal_account_m      17908 non-null int64
personal_account_y      17908 non-null int64
has_debt                17908 non-null int64
amount_requested        1790

In [ ]:

pp.ProfileReport(data)

Number of variables,21
Number of observations,17908
Total Missing (%),0.0%
Total size in memory,2.9 MiB
Average record size in memory,168.0 B
Numeric,17
Categorical,1
Boolean,3
Date,0
Text (Unique),0
Rejected,0


In [ ]:
data.shape

(17908, 21)

In [ ]:
data.isnull().sum().sort_values(ascending=False).head()

e_signed              0
personal_account_y    0
age                   0
pay_schedule          0
home_owner            0
dtype: int64

#### Observations

* 17908 samples, 21 features
* no null values
* Numeric	17
* Categorical	1
* Boolean	3
* 53.8% e-signed the loan

#### EDA

In [ ]:
data.dtypes

entry_id                  int64
age                       int64
pay_schedule             object
home_owner                int64
income                    int64
months_employed           int64
years_employed            int64
current_address_year      int64
personal_account_m        int64
personal_account_y        int64
has_debt                  int64
amount_requested          int64
risk_score                int64
risk_score_2            float64
risk_score_3            float64
risk_score_4            float64
risk_score_5            float64
ext_quality_score       float64
ext_quality_score_2     float64
inquiries_last_month      int64
e_signed                  int64
dtype: object

In [ ]:
data.columns

Index(['entry_id', 'age', 'pay_schedule', 'home_owner', 'income',
       'months_employed', 'years_employed', 'current_address_year',
       'personal_account_m', 'personal_account_y', 'has_debt',
       'amount_requested', 'risk_score', 'risk_score_2', 'risk_score_3',
       'risk_score_4', 'risk_score_5', 'ext_quality_score',
       'ext_quality_score_2', 'inquiries_last_month', 'e_signed'],
      dtype='object')

In [ ]:
data_hist = data.drop(['entry_id', 'pay_schedule'], axis = 1)

In [ ]:
data_hist.columns

Index(['age', 'home_owner', 'income', 'months_employed', 'years_employed',
       'current_address_year', 'personal_account_m', 'personal_account_y',
       'has_debt', 'amount_requested', 'risk_score', 'risk_score_2',
       'risk_score_3', 'risk_score_4', 'risk_score_5', 'ext_quality_score',
       'ext_quality_score_2', 'inquiries_last_month', 'e_signed'],
      dtype='object')

In [ ]:
def num_col_hist(df):
    plt.figure(figsize=(20,10))
    for i in range(1,df.shape[1]+1):
        f = plt.gca()
        col = df.columns.tolist()[i - 1]
        f.set_title(col)
        val = df[col].nunique()
        if val>=100:
            val = 100
        plt.hist(df[col], bins = val)  
        plt.show()

In [ ]:
num_col_hist(data_hist)

**Histogram results shows good.Most of the features are right skewed. But 'Months_employed'shows that the data cured for it is not accurate as most of data shows towords 0 - 1.**

In [ ]:
# Binary columns Pie-chart
#Input - dataframe, binary_col_list, num_rows, num_col

def binary_col_pie(df, binary_col_list, num_rows, num_col):
    fig = plt.figure(figsize=(20,17))
    data = df[binary_col_list]
    for i in range(1, data.shape[1]+1):
        plt.subplot(num_rows,num_col,i)
        f = plt.gca()
        col = data.columns.tolist()[i-1]
        data[col].value_counts().plot.pie(autopct='%1.1f%%')

In [ ]:
binary_col_list = data.nunique()[data.nunique() < 10].index.tolist()
binary_col_list

['pay_schedule', 'home_owner', 'has_debt', 'e_signed']

In [ ]:
binary_col_pie(data, binary_col_list, 2, 2)

**No abnormal distribution of data in Bi/Multi nomial categorical data**

In [ ]:
data.columns

Index(['entry_id', 'age', 'pay_schedule', 'home_owner', 'income',
       'months_employed', 'years_employed', 'current_address_year',
       'personal_account_m', 'personal_account_y', 'has_debt',
       'amount_requested', 'risk_score', 'risk_score_2', 'risk_score_3',
       'risk_score_4', 'risk_score_5', 'ext_quality_score',
       'ext_quality_score_2', 'inquiries_last_month', 'e_signed'],
      dtype='object')

In [ ]:
data2 = data.drop(['entry_id', 'e_signed'], axis = 1)
data2.corrwith(data['e_signed']).plot.bar(figsize=(12,7), title = 'Correlation with Response variable', rot = 45, grid = True)

In [ ]:
data_corr = data2.corrwith(data['e_signed']).rename_axis('Features').reset_index(name='correlation')
data_corr = data_corr.sort_values(by = 'correlation', ascending = False)
Neg_top3 = data_corr.tail(3)['Features'].values.tolist()
pos_top3 = data_corr.head(3)['Features'].values.tolist()

In [ ]:
print('Top 3 postivity correlated feaures with Traget is ',pos_top3)

Top 3 postivity correlated feaures with Traget is  ['risk_score', 'amount_requested', 'has_debt']


In [ ]:
print('Top 3 Negatively correlated feaures with Traget is ',Neg_top3)

Top 3 Negatively correlated feaures with Traget is  ['home_owner', 'personal_account_m', 'age']


In [ ]:
def corr_matrix(df):
    corr = df.corr()
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    
    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(12, 10))
    f.suptitle("Correlation Matrix", fontsize = 25)
    
    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    
    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot_kws={"size": 12})

In [ ]:
corr_matrix(data)

#### Correlation Matrix

* Correlation Matrix helps to identify the most correlated features.If 2 features are highly corelated then we can keep only one and remove other to overcome multicollinearity.
* In the above results the scale shows from -0.2 to +0.3 which is not too high. So most of the features are not high correlated.
* income is correlated with amount_requested
* years_employed is correlated with current_address_year
* amount_requested is correlated with risk
* risk 2,3,4 are correlated with each other

#### Feature Engineering

In [ ]:
data.columns

Index(['entry_id', 'age', 'pay_schedule', 'home_owner', 'income',
       'months_employed', 'years_employed', 'current_address_year',
       'personal_account_m', 'personal_account_y', 'has_debt',
       'amount_requested', 'risk_score', 'risk_score_2', 'risk_score_3',
       'risk_score_4', 'risk_score_5', 'ext_quality_score',
       'ext_quality_score_2', 'inquiries_last_month', 'e_signed'],
      dtype='object')

In [ ]:
# Remove the months_employed column as this is not accurate
# Combine personal_account_m & personal_account_y 
data = data.drop('months_employed', axis = 1)
data['personal_account_months'] = (data['personal_account_m'] + (data['personal_account_y']*12))
data = data.drop(['personal_account_m', 'personal_account_y'], axis = 1)
data.columns

Index(['entry_id', 'age', 'pay_schedule', 'home_owner', 'income',
       'years_employed', 'current_address_year', 'has_debt',
       'amount_requested', 'risk_score', 'risk_score_2', 'risk_score_3',
       'risk_score_4', 'risk_score_5', 'ext_quality_score',
       'ext_quality_score_2', 'inquiries_last_month', 'e_signed',
       'personal_account_months'],
      dtype='object')

In [ ]:
# Convert categorial features to numeric by pd.get_dummies
data = pd.get_dummies(data, drop_first=True)

In [ ]:
len(data.columns)

21

In [ ]:
X = data.drop(['entry_id', 'e_signed'], axis = 1)
y = data['e_signed']

In [ ]:
userid = data['entry_id']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=0, stratify = y)

In [ ]:
y_tr = pd.DataFrame(y_train)
y_ts = pd.DataFrame(y_test)

In [ ]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
print(y_tr['e_signed'].value_counts())
y_tr['e_signed'].value_counts().plot.bar()
plt.subplot(1,2,2)
print(y_tr['e_signed'].value_counts())
y_ts['e_signed'].value_counts().plot.bar()
plt.show()

1    7711
0    6615
Name: e_signed, dtype: int64
1    7711
0    6615
Name: e_signed, dtype: int64


In [ ]:
X_train

,age,home_owner,income,years_employed,current_address_year,has_debt,amount_requested,risk_score,risk_score_2,risk_score_3,risk_score_4,risk_score_5,ext_quality_score,ext_quality_score_2,inquiries_last_month,personal_account_months,pay_schedule_monthly,pay_schedule_semi-monthly,pay_schedule_weekly
12557,56,0,1955,4,2,1,600,62300,0.733666,0.987790,0.491034,0.640974,0.482751,0.382751,3,98,1,0,0
14424,50,1,6500,6,7,1,2800,73900,0.701094,0.910206,0.607794,0.638682,0.368179,0.668179,6,42,0,0,0
3330,54,1,5180,7,7,1,900,77200,0.573031,0.896918,0.498791,0.619568,0.542286,0.842286,2,39,0,0,0
12300,39,1,2022,1,3,1,500,57000,0.608957,0.890173,0.604732,0.613718,0.683179,0.383179,2,54,0,0,1
6068,57,1,3200,4,1,0,1600,72200,0.705163,0.754366,0.468130,0.498718,0.650596,0.650596,6,38,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,55,0,4165,3,0,1,425,52700,0.677441,0.872183,0.610830,0.843005,0.770093,0.470093,6,27,0,0,1
587,25,1,4010,1,0,0,1000,61800,0.796318,0.874282,0.637565,0.630028,0.648853,0.448853,8,38,0,0,0
709,30,0,3800,3,2,1,700,56900,0.752629,0.886683,0.593959,0.761297,0.845579,0.645579,7,40,0,0,1
17303,33,0,3630,2,4,1,700,43750,0.642991,0.933853,0.750195,0.834345,0.807275,0.707275,5,37,0,0,1


In [ ]:
# Apply standard scaling to the X_train & X_test

from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
X_train2 = pd.DataFrame(sc_x.fit_transform(X_train))
X_test2 = pd.DataFrame(sc_x.transform(X_test))

In [ ]:
X_train2.columns = X_train.columns.values
X_test2.columns = X_test.columns.values
X_train2.index = X_train.index.values
X_test2.index = X_test.index.values

X_train = X_train2
X_test = X_test2

In [ ]:
X_train

,age,home_owner,income,years_employed,current_address_year,has_debt,amount_requested,risk_score,risk_score_2,risk_score_3,risk_score_4,risk_score_5,ext_quality_score,ext_quality_score_2,inquiries_last_month,personal_account_months,pay_schedule_monthly,pay_schedule_semi-monthly,pay_schedule_weekly
12557,1.094198,-0.862979,-1.137425,0.207337,-0.577714,0.506840,-0.506149,0.077451,0.477309,2.017041,-0.734759,-0.645764,-1.004049,-1.717506,-0.935504,2.271741,3.280534,-0.351955,-0.511623
14424,0.587988,1.158777,1.886386,1.093997,1.238705,0.506840,2.703968,0.831560,0.117112,0.585978,0.199560,-0.664884,-1.821716,0.330542,-0.123998,-0.143025,-0.304828,-0.351955,-0.511623
3330,0.925461,1.158777,1.008183,1.537326,1.238705,0.506840,-0.068406,1.046091,-1.299098,0.340877,-0.672686,-0.824316,-0.579165,1.579821,-1.206006,-0.272387,-0.304828,-0.351955,-0.511623
12300,-0.340065,1.158777,-1.092850,-1.122652,-0.214430,0.506840,-0.652063,-0.267099,-0.901802,0.216464,0.175060,-0.873106,0.426347,-1.714435,-1.206006,0.374425,-0.304828,-0.351955,1.954565
6068,1.178566,1.158777,-0.309121,0.207337,-0.940998,-1.973008,0.952995,0.721044,0.162109,-2.288570,-0.918044,-1.832334,0.193811,0.204377,-0.123998,-0.315508,-0.304828,2.841273,-0.511623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,1.009830,-0.862979,0.332899,-0.235993,-1.304281,0.506840,-0.761499,-0.546640,-0.144459,-0.115374,0.223860,1.039391,1.046627,-1.090796,-0.123998,-0.789837,-0.304828,-0.351955,1.954565
587,-1.521222,1.158777,0.229776,-1.122652,-1.304281,-1.973008,0.077509,0.044946,1.170159,-0.076663,0.437791,-0.737070,0.181372,-1.243200,0.417007,-0.315508,-0.304828,-0.351955,-0.511623
709,-1.099380,-0.862979,0.090062,-0.235993,-0.577714,0.506840,-0.360234,-0.273600,0.687025,0.152084,0.088856,0.357858,1.585348,0.168379,0.146505,-0.229266,-0.304828,-0.351955,1.954565
17303,-0.846275,-0.862979,-0.023039,-0.679323,0.148854,0.506840,-0.360234,-1.128474,-0.525435,1.022149,1.339069,0.967155,1.311984,0.611069,-0.394500,-0.358629,-0.304828,-0.351955,1.954565


#### Machine Learning 

##### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
classifer = LogisticRegression(penalty = 'l1', random_state = 0)

In [ ]:
classifer.fit(X_train, y_train)
y_pred = classifer.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [ ]:
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

results = pd.DataFrame([['Logistic regression(Lasso)', acc, prec, rec, f1]], 
                             columns=['model', 'acc', 'prec', 'rec', 'f1'])



In [ ]:
results

,model,acc,prec,rec,f1
0,Logistic regression(Lasso),0.581798,0.588916,0.738589,0.655315


* **Precision tells us about the success probability of making a correct positive class classification**
* **Recall explains how sensitive the model is towards identifying the positive class**
* **Along with Accuracy, Percision and recall scores are important in assesing the model**
* **Classification model should have low FP & FN compared to TP**

##### ## SVM (Linear)

In [ ]:
from sklearn.svm import SVC
classifer = SVC(kernel='linear', random_state = 0)

classifer.fit(X_train, y_train)
y_pred = classifer.predict(X_test)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

results = results.append({'model':'SVC (Linear)', 'acc' : acc, 'prec' : prec, 'rec': rec, 'f1': f1}, ignore_index=True)

In [ ]:
from sklearn.svm import SVC
classifer = SVC(kernel='rbf', random_state = 0)

classifer.fit(X_train, y_train)
y_pred = classifer.predict(X_test)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

results = results.append({'model':'SVC (rbf)', 'acc' : acc, 'prec' : prec, 'rec': rec, 'f1': f1}, ignore_index=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifer = RandomForestClassifier(n_estimators = 100,
                                    criterion = 'entropy', random_state = 0)

classifer.fit(X_train, y_train)
y_pred = classifer.predict(X_test)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

results = results.append({'model':'Random Forest( Entropy)', 'acc' : acc, 'prec' : prec, 'rec': rec, 'f1': f1}, ignore_index=True)

In [ ]:
results

,model,acc,prec,rec,f1
0,Logistic regression(Lasso),0.581798,0.588916,0.738589,0.655315
1,SVC (Linear),0.586544,0.589723,0.761929,0.664856
2,SVC (rbf),0.615578,0.622173,0.727697,0.670810
3,Random Forest( Entropy),0.636237,0.654780,0.685685,0.669876


In [ ]:
# Cross-validation 

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifer, X = X_train, y = y_train, cv=10)
print("Random Forest Classifier Accuracy: %0.2f (+/- %0.2f)"  % (accuracies.mean(), accuracies.std() * 2))

Random Forest Classifier Accuracy: 0.63 (+/- 0.02)


**Random Forest Classifier Accuracy of 63% is consistent**

In [ ]:
!pip install TPOT
from tpot import TPOTClassifier


In [ ]:
tpot = TPOTClassifier(verbosity=2, scoring="accuracy", n_jobs=-1, max_time_mins=20, max_eval_time_mins=2,
                      population_size=10, early_stop=5)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

Generation 1 - Current best internal CV score: 0.6296243251504976
Generation 2 - Current best internal CV score: 0.6296243251504976
Generation 3 - Current best internal CV score: 0.6296243251504976

The optimized pipeline was not improved after evaluating 5 more generations. Will end the optimization process.

TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: GradientBoostingClassifier(input_matrix, learning_rate=0.01, max_depth=8, max_features=0.6500000000000001, min_samples_leaf=15, min_samples_split=5, n_estimators=100, subsample=0.7500000000000001)
0.6395868230039085


In [ ]:
tpot.fitted_pipeline_

Pipeline(memory=None,
         steps=[('gradientboostingclassifier',
                 GradientBoostingClassifier(criterion='friedman_mse', init=None,
                                            learning_rate=0.01, loss='deviance',
                                            max_depth=8,
                                            max_features=0.6500000000000001,
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.0,
                                            min_impurity_split=None,
                                            min_samples_leaf=15,
                                            min_samples_split=5,
                                            min_weight_fraction_leaf=0.0,
                                            n_estimators=100,
                                            n_iter_no_change=None,
                                            presort='auto', random_state=None,
                  

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
classifer = GradientBoostingClassifier(criterion='friedman_mse', init=None,
                                            learning_rate=0.01, loss='deviance',
                                            max_depth=8,
                                            max_features=0.6500000000000001,
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.0,
                                            min_impurity_split=None,
                                            min_samples_leaf=15,
                                            min_samples_split=5,
                                            min_weight_fraction_leaf=0.0,
                                            n_estimators=100,
                                            n_iter_no_change=None,
                                            presort='auto', random_state=None,
                                            subsample=0.7500000000000001,
                                            tol=0.0001, validation_fraction=0.1,
                                            verbose=0, warm_start=False)

In [ ]:
classifer.fit(X_train, y_train)
y_pred = classifer.predict(X_test)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

results = results.append({'model':'GradientBoostingClassifier(TPOT)', 'acc' : acc, 'prec' : prec, 'rec': rec, 'f1': f1}, ignore_index=True)

In [ ]:
results

,model,acc,prec,rec,f1
0,Logistic regression(Lasso),0.581798,0.588916,0.738589,0.655315
1,SVC (Linear),0.586544,0.589723,0.761929,0.664856
2,SVC (rbf),0.615578,0.622173,0.727697,0.670810
3,Random Forest( Entropy),0.636237,0.654780,0.685685,0.669876
4,GradientBoostingClassifier(TPOT),0.637912,0.636050,0.765041,0.694608


In [ ]:
tpot = TPOTClassifier(verbosity=2, scoring="accuracy", n_jobs=-1, max_time_mins=60, max_eval_time_mins=2,
                      population_size=10)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

Generation 1 - Current best internal CV score: 0.630182740659279
Generation 2 - Current best internal CV score: 0.630182740659279
Generation 3 - Current best internal CV score: 0.6308804555789792
Generation 4 - Current best internal CV score: 0.6402340006016254
Generation 5 - Current best internal CV score: 0.6402340006016254
Generation 6 - Current best internal CV score: 0.6402340006016254
Generation 7 - Current best internal CV score: 0.6421186712117393
Generation 8 - Current best internal CV score: 0.6426069376903676
Generation 9 - Current best internal CV score: 0.6426069376903676
Generation 10 - Current best internal CV score: 0.6426069376903676
Generation 11 - Current best internal CV score: 0.6426069376903676
Generation 12 - Current best internal CV score: 0.6426069376903676
Generation 13 - Current best internal CV score: 0.6426069376903676
Generation 14 - Current best internal CV score: 0.6426069376903676
Generation 15 - Current best internal CV score: 0.6426069376903676
Genera

In [ ]:
tpot.fitted_pipeline_

Pipeline(memory=None,
         steps=[('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=5,
                               min_child_weight=3, missing=None,
                               n_estimators=100, n_jobs=1, nthread=1,
                               objective='binary:logistic', random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               seed=None, silent=None, subsample=0.8,
                               verbosity=1))],
         verbose=False)

In [ ]:
from xgboost import XGBClassifier
classifer = XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=5,
                               min_child_weight=3, missing=None,
                               n_estimators=100, n_jobs=1, nthread=1,
                               objective='binary:logistic', random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               seed=None, silent=None, subsample=0.8,
                               verbosity=1)

In [ ]:
classifer.fit(X_train, y_train)
y_pred = classifer.predict(X_test)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

results = results.append({'model':'XGBClassifier(TPOT - 60min)', 'acc' : acc, 'prec' : prec, 'rec': rec, 'f1': f1}, ignore_index=True)

In [ ]:
results

,model,acc,prec,rec,f1
0,Logistic regression(Lasso),0.581798,0.588916,0.738589,0.655315
1,SVC (Linear),0.586544,0.589723,0.761929,0.664856
2,SVC (rbf),0.615578,0.622173,0.727697,0.670810
3,Random Forest( Entropy),0.636237,0.654780,0.685685,0.669876
4,GradientBoostingClassifier(TPOT),0.637912,0.636050,0.765041,0.694608
5,XGBClassifier(TPOT - 60min),0.642379,0.656659,0.703320,0.679189
